In [1]:
# Project
# github restricts file size, dataset can be downloaded from https://www.kaggle.com/gyani95/380000-lyrics-from-metrolyrics/data


In [2]:
# import the dataset
import io
import pandas as pd

data = pd.read_csv('lyrics.csv')
data.head()

# temp so that it runs during testing
data = data[:10000]

/Users/nathanjenkins/opt/anaconda3/envs/mining/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
# preprocess the data

# remove all the genres with not avaliable and other
data = data[data.genre != 'Not Available']
data = data[data.genre != 'Other']


# remove all the data with no lyrics
data.dropna(subset=['lyrics'], inplace=True)
print(data.shape)

import nltk
from nltk.corpus import stopwords

(6212, 6)
/Users/nathanjenkins/opt/anaconda3/envs/mining/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [4]:
from sklearn.feature_extraction.text import CountVectorizer

# max_features : a maximum of 1500 columns
# min_df : the word must occur in a mininum of 5 documents
# max_df : the word occurs in less than 70 percent of the documents (words appearing in all are useless)
# nltk.download('stopwords')
vectorizer = CountVectorizer(max_features=1500, min_df=2, max_df=0.7, stop_words=stopwords.words('english'))

# creates a bag of words
X = vectorizer.fit_transform(data.lyrics.values.astype('str'))

# creates y, which is our genre class
y = data.genre

# visualize the transformed data 
pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

,10,2x,40,50,across,act,admit,afraid,age,ago,...,îµî,îµî¹ï,îµï,î¹,î¹î,îºî,î¼î,î¼îµ,î½,î½î
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6207,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6208,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6209,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6210,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
,# find the TFID reference: http://www.tfidf.com/
# mitigates the fact that the word may have a high frequency in other documents

# TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
# IDF(t) = log_e(Total number of documents / Number of documents with term t in it). <-- appro 
    # if all documents have the term in it then it has a score of 0 TF * log(1) = 0
    # it will weigh rare words heigher 

# TFIDF = TF * IDF

from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X)

In [6]:
# notice how the data is now altered based on the word frequency
pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

,10,2x,40,50,across,act,admit,afraid,age,ago,...,îµî,îµî¹ï,îµï,î¹,î¹î,îºî,î¼î,î¼îµ,î½,î½î
0,0.0,0.0,0.0,0.0,0.0,0.0,0.067852,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6207,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6208,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6209,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6210,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# split X and y into training a test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)


In [16]:

def train_predict(classifiers):
    for classifier in classifiers:
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)
        print(classifier.__class__.__name__, accuracy_score(y_test, y_pred))
        print(classification_report(y_test, y_pred))


In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, classification_report

classifers = [RandomForestClassifier(), MultinomialNB(), LogisticRegression()]
train_predict(classifers)

/Users/nathanjenkins/opt/anaconda3/envs/mining/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
RandomForestClassifier 0.5872504829362524
              precision    recall  f1-score   support

     Country       0.25      0.02      0.04        45
  Electronic       0.64      0.24      0.35        66
        Folk       0.40      0.06      0.10        36
     Hip-Hop       0.84      0.82      0.83       203
       Indie       1.00      0.17      0.29        30
        Jazz       0.43      0.11      0.17        83
       Metal       0.54      0.47      0.50       175
         Pop       0.42      0.25      0.31       222
         R&B       0.00      0.00      0.00         4
        Rock       0.57      0.83      0.68       689

    accuracy                           0.59      1553
   macro avg       0.51      0.30      0.33  